# iNaturalist status updates by state - QLD


Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist [taxa list](#inaturalist-taxonomy)
3. Read in the state sensitive and conservation list, concatenate them into a single list
4. Wash the names in the state list through the gbif name parser
5. Attempt to match the state statuses to an IUCN equivalent
6. Determine the best placeID to use for this state

## Next steps:
7. Find Updates and Additions
7.1 Left join the state list with the iNaturalist statuses on scientificName
  * **Match** UPDATE the status (new details, new dept name or url)
  * **No Match** Left join the remainder (noinatstatus) to the inat taxonomy
     * Yes - ADD new status record
     * No - REPORT. Seek synonyms for the taxon, or create species in iNat for critical species

8. Find [Removals](##removals) - Left join the inaturalist statuses with the update list. Report on the remainder.



# iNaturalist statuses

In [1]:
import pandas as pd
import sys
import os
projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
#projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [2]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])

inatstatuses = filter_state_statuses("Qld|QLD|Queensland|QUEENSLAND|QL", ".qld.")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
532,223098,1003032,3669610,7308,,Queensland Nature Conservation Act 1992,CE,https://apps.des.qld.gov.au/species-search/det...,,obscured,...,Antrophyum,austroqueenslandicum,,2021-10-01T10:37:36Z,Antrophyum austroqueenslandicum,species,,,,
131,169944,1004434,58320,7308,,Queensland Nature Conservation Act 1992,Least concern,https://apps.des.qld.gov.au/species-search/det...,Listed as 'confidential' by the Queensland Dep...,obscured,...,Oberonia,crateriformis,,2021-07-18T09:05:24Z,Oberonia crateriformis,species,http://www.catalogueoflife.org/annual-checklis...,,,
1473,152757,100615,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,bindal,,2020-05-28T05:44:48Z,Euastacus bindal,species,http://www.iucnredlist.org/apps/redlist/details,,,
1407,152668,100635,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,jagara,,2020-05-28T05:43:40Z,Euastacus jagara,species,http://www.iucnredlist.org/apps/redlist/details,,,
1267,152461,100638,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,maidae,,2020-05-28T05:06:01Z,Euastacus maidae,species,http://www.iucnredlist.org/apps/redlist/details,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,262047,977629,3669610,7308,,Queensland Nature Conservation Act 1992,NT,https://www.data.qld.gov.au/dataset/conservati...,,open,...,Melaleuca,formosa,,2022-09-05T10:47:52Z,Melaleuca formosa,species,,,,
2615,161807,993333,702203,7308,,Nature Conservation Act 1992,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
2616,161808,993333,702203,6744,,Environment Protection and Biodiversity Conser...,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
23,167723,993605,3669610,7308,,QLD DEHP,NT,https://www.data.qld.gov.au/dataset/conservati...,,obscured,...,Acianthus,amplexicaulis,,2021-10-05T08:48:02Z,Acianthus amplexicaulis,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [3]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


# State lists

Sensitive list: `geoprivacy` = `obscured`
overrides


In [4]:
sensitivelist = lf.download_ala_list("https://lists-test.ala.org.au/ws/speciesListItems/dr18404?max=10000&includeKVP=true")
sensitivelist = lf.kvp_to_columns(sensitivelist)
sensitivelist['name'] = sensitivelist['name'].str.replace('subsp. ', '', regex=False)
sensitivelist = sensitivelist.rename(columns={'taxonID':'wildnetTaxonID'})
sensitivelist['wildnet_geoprivacy'] = 'obscured'
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,wildnetTaxonID,scientificNameAuthorship,sourceStatus,Significant,status,Endemicity,E P B C Status,wildnet_geoprivacy
0,2799651,Rhinolophus philippinensis,Greater Large-eared Horseshoe Bat,Rhinolophus philippinensis,ALA_DR652_255,dr18404,"[{'key': 'taxonID', 'value': '969'}, {'key': '...",969,"Waterhouse, 1843",E,Y,Endangered,Regional Endemic,Vulnerable,obscured
0,2799225,Chloebia gouldiae,Gouldian Finch,Chloebia gouldiae,https://biodiversity.org.au/afd/taxa/27d27be2-...,dr18404,"[{'key': 'taxonID', 'value': '1376'}, {'key': ...",1376,"(Gould, 1844)",E,Y,Endangered,Intranational,Endangered,obscured
0,2798738,Erythrura trichroa,Blue-faced Parrot-finch,Erythrura (Acalanthe) trichroa,https://biodiversity.org.au/afd/taxa/cb632014-...,dr18404,"[{'key': 'taxonID', 'value': '1378'}, {'key': ...",1378,"(Kittlitz, 1835)",NT,Y,Near Threatened,Not Endemic to Australia,NaN,obscured
0,2799209,Neochmia phaeton evangelinae,None,Neochmia (Neochmia) phaeton evangelinae,https://biodiversity.org.au/afd/taxa/565b6f37-...,dr18404,"[{'key': 'taxonID', 'value': '1370'}, {'key': ...",1370,"(Hombron & Jacquinot, 1841)",E,Y,Endangered,Regional Endemic,Endangered,obscured
0,2799217,Poephila cincta cincta,Black-throated Finch (southern Subspecies),Poephila (Poephila) cincta cincta,https://biodiversity.org.au/afd/taxa/9fe878db-...,dr18404,"[{'key': 'taxonID', 'value': '1365'}, {'key': ...",1365,"Gould, 1837",E,Y,Endangered,Intranational,Endangered,obscured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2799125,Pneumatopteris costata,None,Pneumatopteris costata,https://id.biodiversity.org.au/node/apni/2887362,dr18404,"[{'key': 'taxonID', 'value': '11699'}, {'key':...",11699,(Brack.) Holttum,NT,Y,Near Threatened,Regional Endemic,NaN,obscured
0,2799126,Pneumatopteris pennigera,Pākau,Pneumatopteris pennigera,https://id.biodiversity.org.au/node/apni/2899796,dr18404,"[{'key': 'taxonID', 'value': '11700'}, {'key':...",11700,(G.Forst.) Holttum,E,Y,Endangered,Not Endemic to Australia,NaN,obscured
0,2799558,Thelypteris confluens,Swamp Fern,Thelypteris confluens,https://id.biodiversity.org.au/node/apni/2913519,dr18404,"[{'key': 'taxonID', 'value': '16042'}, {'key':...",16042,(Thunb.) C.V.Morton,V,Y,Vulnerable,Not Endemic to Australia,NaN,obscured
0,2799401,Macadamia jansenii,Bulburin Nut Tree,Macadamia jansenii,https://id.biodiversity.org.au/node/apni/2904937,dr18404,"[{'key': 'taxonID', 'value': '8185'}, {'key': ...",8185,C.L.Gross & P.H.Weston,CR,Y,Critically Endangered,Queensland Endemic,Endangered,obscured


In [5]:
conservationlist = lf.download_ala_list("https://lists-test.ala.org.au/ws/speciesListItems/dr652?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist['name'] = conservationlist['name'].str.replace('subsp. ', '', regex=False)
conservationlist = conservationlist.rename(columns={'taxonID':'wildnetTaxonID'})
conservationlist['wildnet_geoprivacy'] = conservationlist['Confidential'].apply(lambda x: 'obscured' if x == 'Y' else 'open')
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,wildnetTaxonID,scientificNameAuthorship,sourceStatus,Significant,Confidential,status,Endemicity,E P B C Status,wildnet_geoprivacy
0,2798372,Adelotus brevis,Tusked Frog,Adelotus brevis,https://biodiversity.org.au/afd/taxa/32f57cc7-...,dr652,"[{'key': 'taxonID', 'value': '706'}, {'key': '...",706,"(Günther, 1863)",V,Y,N,Vulnerable,Intranational,NaN,open
0,2796770,Philoria kundagungan,Mountain Frog,Philoria kundagungan,https://biodiversity.org.au/afd/taxa/fcb2e700-...,dr652,"[{'key': 'taxonID', 'value': '687'}, {'key': '...",687,"(Ingram & Corben, 1975)",E,Y,Y,Endangered,Intranational,Endangered,obscured
0,2796819,Crinia tinnula,Wallum Froglet,Crinia tinnula,https://biodiversity.org.au/afd/taxa/a101af4a-...,dr652,"[{'key': 'taxonID', 'value': '686'}, {'key': '...",686,"Straughan & Main, 1966",V,Y,N,Vulnerable,Intranational,NaN,open
0,2797552,Mixophyes fleayi,Fleay's Barred Frog,Mixophyes fleayi,https://biodiversity.org.au/afd/taxa/289e1b7c-...,dr652,"[{'key': 'taxonID', 'value': '675'}, {'key': '...",675,"Corben & Ingram, 1987",E,Y,Y,Endangered,Intranational,Endangered,obscured
0,2796637,Mixophyes iteratus,Giant Barred Frog,Mixophyes iteratus,https://biodiversity.org.au/afd/taxa/9eef5215-...,dr652,"[{'key': 'taxonID', 'value': '676'}, {'key': '...",676,"Straughan, 1968",V,Y,Y,Vulnerable,Intranational,Vulnerable,obscured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2797742,Macrozamia viridis,None,Macrozamia viridis,https://id.biodiversity.org.au/node/apni/2917053,dr652,"[{'key': 'taxonID', 'value': '6482'}, {'key': ...",6482,D.L.Jones & P.I.Forst.,E,Y,Y,Endangered,Intranational,NaN,obscured
0,2796733,Alpinia hylandii,None,Alpinia hylandii,https://id.biodiversity.org.au/node/apni/2907514,dr652,"[{'key': 'taxonID', 'value': '8948'}, {'key': ...",8948,R.M.Sm.,NT,Y,N,Near Threatened,Queensland Endemic,NaN,open
0,2797715,Amomum queenslandicum,None,Amomum queenslandicum,https://id.biodiversity.org.au/node/apni/2893545,dr652,"[{'key': 'taxonID', 'value': '8949'}, {'key': ...",8949,R.M.Sm.,V,Y,N,Vulnerable,Queensland Endemic,NaN,open
0,2797487,Globba marantina,Kera Kera,Globba marantina,https://id.biodiversity.org.au/node/apni/2901385,dr652,"[{'key': 'taxonID', 'value': '12434'}, {'key':...",12434,L.,V,Y,N,Vulnerable,Regional Endemic,NaN,open


In [6]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['wildnetTaxonID', 'name', 'status', 'wildnet_geoprivacy', 'lsid']],
                    conservationlist[['wildnetTaxonID', 'name', 'status', 'wildnet_geoprivacy', 'lsid']]]).drop_duplicates()

In [7]:
%%script echo skipping # comment this line to retrieve binomial and trinomial names from GBIF
parsednames = lf.gbifparse(statelist)
parsednames.to_csv(sourcedir + "qld-gbif.csv", index=False)

skipping # comment this line to retrieve binomial and trinomial names from GBIF


In [8]:
parsednames = pd.read_csv(sourcedir + "qld-gbif.csv")
statelist = statelist.merge(parsednames[['scientificName','canonicalName','canonicalNameComplete','type','rankMarker']],how="left",left_on="name",right_on="scientificName")
numfullstatelist = len(statelist.index)
statelist['scientificName'] = statelist['canonicalName']
statelist

,wildnetTaxonID,name,status,wildnet_geoprivacy,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker
0,969,Rhinolophus philippinensis,Endangered,obscured,ALA_DR652_255,Rhinolophus philippinensis,Rhinolophus philippinensis,Rhinolophus philippinensis,SCIENTIFIC,sp.
1,1376,Chloebia gouldiae,Endangered,obscured,https://biodiversity.org.au/afd/taxa/27d27be2-...,Chloebia gouldiae,Chloebia gouldiae,Chloebia gouldiae,SCIENTIFIC,sp.
2,1378,Erythrura trichroa,Near Threatened,obscured,https://biodiversity.org.au/afd/taxa/cb632014-...,Erythrura trichroa,Erythrura trichroa,Erythrura trichroa,SCIENTIFIC,sp.
3,1370,Neochmia phaeton evangelinae,Endangered,obscured,https://biodiversity.org.au/afd/taxa/565b6f37-...,Neochmia phaeton evangelinae,Neochmia phaeton evangelinae,Neochmia phaeton evangelinae,SCIENTIFIC,infrasp.
4,1365,Poephila cincta cincta,Endangered,obscured,https://biodiversity.org.au/afd/taxa/9fe878db-...,Poephila cincta cincta,Poephila cincta cincta,Poephila cincta cincta,SCIENTIFIC,infrasp.
...,...,...,...,...,...,...,...,...,...,...
2512,6296,Xanthorrhoea sp. (Cape Bedford M.Gandini AQ601...,Special least concern,open,https://id.biodiversity.org.au/node/apni/2906735,Xanthorrhoea spec.,Xanthorrhoea spec.,Xanthorrhoea spec.,INFORMAL,sp.
2513,8948,Alpinia hylandii,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2907514,Alpinia hylandii,Alpinia hylandii,Alpinia hylandii,SCIENTIFIC,sp.
2514,8949,Amomum queenslandicum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2893545,Amomum queenslandicum,Amomum queenslandicum,Amomum queenslandicum,SCIENTIFIC,sp.
2515,12434,Globba marantina,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901385,Globba marantina,Globba marantina,Globba marantina,SCIENTIFIC,sp.


In [9]:
# Identify records that won't comply with iNaturalist species names - print these later
noncomply = statelist[statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID']) ]
# remove records that do not comply
statelist = statelist[~statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID']) ]
statelist = pd.DataFrame(statelist[['wildnetTaxonID','scientificName','status','wildnet_geoprivacy','lsid']])
statelist

,wildnetTaxonID,scientificName,status,wildnet_geoprivacy,lsid
0,969,Rhinolophus philippinensis,Endangered,obscured,ALA_DR652_255
1,1376,Chloebia gouldiae,Endangered,obscured,https://biodiversity.org.au/afd/taxa/27d27be2-...
2,1378,Erythrura trichroa,Near Threatened,obscured,https://biodiversity.org.au/afd/taxa/cb632014-...
3,1370,Neochmia phaeton evangelinae,Endangered,obscured,https://biodiversity.org.au/afd/taxa/565b6f37-...
4,1365,Poephila cincta cincta,Endangered,obscured,https://biodiversity.org.au/afd/taxa/9fe878db-...
...,...,...,...,...,...
2511,12306,Xanthorrhoea pumilio,Special least concern,open,https://id.biodiversity.org.au/node/apni/2916574
2513,8948,Alpinia hylandii,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2907514
2514,8949,Amomum queenslandicum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2893545
2515,12434,Globba marantina,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901385


In [10]:
# check for duplicates with conflicting information
dupinformation = statelist.groupby('wildnetTaxonID').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
dupinformation

,wildnetTaxonID,scientificName,status,wildnet_geoprivacy,lsid


### 4. Equivalent IUCN statuses

In [11]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild','Extinct'}
statelist.groupby(['status'])['status'].count()

status
Critically Endangered    132
Endangered               288
Extinct                    5
Extinct in the wild       26
Least concern            132
Near Threatened          273
Special least concern    900
Vulnerable               593
Name: status, dtype: int64

In [12]:
# iucnStatusMappings = {

iucnStatusMappings = {
    'Least concern':'Least Concern',
    'Special least concern':'Least Concern',
    'Extinct in the wild':'Extinct in the Wild'
}

### 5. Determine best place ID to use

In [13]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 7308

place_id  place_name             place_display_name           
                                                                    5
144315    Brisbane City          Brisbane City                      2
153119    South East Queensland  South East Queensland, QL, AU      1
18870     Cairns - Pt B          Cairns - Pt B, QL, AU              1
19232     Yarrabah               Yarrabah, QL, AU                   1
6744      Australia              Australia                         12
7308      Queensland             Queensland, AU                   631
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State lists on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [14]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['wildnetTaxonID','scientificName','status','wildnet_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,wildnetTaxonID,scientificName,status,wildnet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
1651,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1743,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1744,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2300,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2301,41054,Zieria wilhelminae,Critically Endangered,open,ALA_DR652_2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2483,14130,Zostera capricorni,Special least concern,open,https://id.biodiversity.org.au/node/apni/2905201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Qld Department of Environment and Science"
mergedstatuses['new_description'] = "Listed as Confidential - refer to https://www.data.qld.gov.au/dataset/queensland-confidential-species"
#mergedstatuses['new_url'] = mergedstatuses['lsid']
mergedstatuses['new_url'] = "https://apps.des.qld.gov.au/species-search/details/?id=" + mergedstatuses['wildnetTaxonID'].astype(str)
mergedstatuses.rename(columns={'wildnet_geoprivacy':'new_geoprivacy'},inplace=True)
mergedstatuses['new_place_id'] = '7308'  # Queensland, AU
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Confidential')
mergedstatuses

,wildnetTaxonID,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,...,geoprivacy,place_id,place_display_name,new_authority,new_description,new_url,new_place_id,new_username,new_iucn_equivalent,new_status
1651,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Special least concern
1652,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Special least concern
1653,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Special least concern
1743,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Near Threatened
1744,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Critically Endangered
2300,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Vulnerable
2301,41054,Zieria wilhelminae,Critically Endangered,open,ALA_DR652_2004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Critically Endangered
2483,14130,Zostera capricorni,Special least concern,open,https://id.biodiversity.org.au/node/apni/2905201,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,7308,peggydnew,Vulnerable,Special least concern


## Updates

In [16]:
# those that need to be updated - we found a status
mergedstatuses[mergedstatuses['status_id'].notnull()][['scientificName','status','status_inat','new_geoprivacy','geoprivacy','authority','user_id']]

,scientificName,status,status_inat,new_geoprivacy,geoprivacy,authority,user_id
1749,Acacia attenuata,Vulnerable,VU,open,,Queensland,702203
1750,Acacia barakulensis,Vulnerable,VU,open,obscured,QLD DEHP,702203
1751,Acacia baueri baueri,Vulnerable,VU,open,,Queensland,702203
1752,Acacia calantha,Near Threatened,NT,open,,Queensland Government,702203
1761,Acacia hockingsii,Vulnerable,VU,open,open,Queensland Nature Conservation Act 1992,3669610
...,...,...,...,...,...,...,...
2283,Zieria bifida,Endangered,EN,open,,Queensland Nature Conservation Act 1992,702203
2286,Zieria collina,Vulnerable,VU,open,open,Nature Conservation Act 1992,702203
2288,Zieria exsul,Critically Endangered,CR,open,,Queensland,702203
2291,Zieria gymnocarpa,Critically Endangered,CR,open,,QLD DEHP,409010


In [17]:
# updates - create the data frame
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
1749,UPDATE,Acacia attenuata,159918,898644,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1750,UPDATE,Acacia barakulensis,158182,1038687,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1751,UPDATE,Acacia baueri baueri,159921,1111700,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1752,UPDATE,Acacia calantha,160798,1121183,Near Threatened,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1761,UPDATE,Acacia hockingsii,264938,1252519,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...
2283,UPDATE,Zieria bifida,223195,1125311,Endangered,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
2286,UPDATE,Zieria collina,262040,537414,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
2288,UPDATE,Zieria exsul,159923,1111558,Critically Endangered,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
2291,UPDATE,Zieria gymnocarpa,169061,1244660,Critically Endangered,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....


## No status in iNaturalist via straight scientificName match
The Qld records that didn't match up to a status in iNaturalist

In [18]:
# to add: those that have no inaturalist status
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,wildnetTaxonID,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Polypodiopsida,Hymenophyllales,Hymenophyllaceae,Abrodictyum,caudatum,NaN,2022-06-07T08:14:14Z,species,NaN
2,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,acrionastes,NaN,2022-04-06T22:12:50Z,species,http://powo.science.kew.org/
4,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ammophila,NaN,2022-04-07T01:03:28Z,species,http://powo.science.kew.org/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,vagans,NaN,2021-07-28T04:47:46Z,species,https://eol.org/pages/52204430
1911,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,verrucosa,NaN,2021-07-28T04:47:50Z,species,https://eol.org/pages/49431650
1912,41054,Zieria wilhelminae,Critically Endangered,open,ALA_DR652_2004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913,14130,Zostera capricorni,Special least concern,open,https://id.biodiversity.org.au/node/apni/2905201,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Alismatales,Zosteraceae,Zostera,capricorni,NaN,2019-02-15T07:39:29Z,species,NaN


In [19]:
noinatstatus[noinatstatus['id'].notna()] # there's no status but there is a matching inat taxon (id is the taxon id)
# note: "Dendrobium" matches to both genus and section

,wildnetTaxonID,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
1,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Polypodiopsida,Hymenophyllales,Hymenophyllaceae,Abrodictyum,caudatum,NaN,2022-06-07T08:14:14Z,species,NaN
3,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,acrionastes,NaN,2022-04-06T22:12:50Z,species,http://powo.science.kew.org/
4,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ammophila,NaN,2022-04-07T01:03:28Z,species,http://powo.science.kew.org/
5,3304,Acacia arbiana,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2906686,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,arbiana,NaN,2022-04-06T22:14:00Z,species,http://powo.science.kew.org/
6,31097,Acacia argentina,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2888490,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,argentina,NaN,2022-04-03T23:48:35Z,species,http://powo.science.kew.org/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,27791,Zieria obovata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2917229,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,obovata,NaN,2020-02-19T19:36:10Z,species,https://eol.org/pages/52204447
1909,9476,Zieria rimulosa,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2913391,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,rimulosa,NaN,2021-07-28T04:47:47Z,species,https://eol.org/pages/52204449
1910,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,vagans,NaN,2021-07-28T04:47:46Z,species,https://eol.org/pages/52204430
1911,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,verrucosa,NaN,2021-07-28T04:47:50Z,species,https://eol.org/pages/49431650


In [20]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
1,ADD,Abrodictyum caudatum,NaN,451374,Special least concern,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
3,ADD,Acacia acrionastes,NaN,898576,Near Threatened,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
4,ADD,Acacia ammophila,NaN,898600,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
5,ADD,Acacia arbiana,NaN,898626,Near Threatened,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
6,ADD,Acacia argentina,NaN,898630,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...
1908,ADD,Zieria obovata,NaN,956303,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1909,ADD,Zieria rimulosa,NaN,1247621,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1910,ADD,Zieria vagans,NaN,1247619,Critically Endangered,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1911,ADD,Zieria verrucosa,NaN,1247620,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,Listed as Confidential - refer to https://www....


In [21]:
# write these to the file
pd.concat([updates, additions]).to_csv(sourcedir + "qld.csv", index=False)

## Reports
### Statuses with no matching taxon in iNaturalist

In [22]:
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat.sort_values(['status','new_geoprivacy'])

,wildnetTaxonID,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
499,21854,Crepidomanes aphlebioides,Critically Endangered,obscured,https://id.biodiversity.org.au/node/apni/7404140,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,34929,Diplazium squamuligerum,Critically Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5129...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720,41407,Drosera buubugujin,Critically Endangered,obscured,ALA_DR652_710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762,29733,Emydura subglobosa angkibaanya,Critically Endangered,obscured,ALA_DR652_302,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,6,Hypochrysops piceatus,Critically Endangered,obscured,https://biodiversity.org.au/afd/taxa/e34f8ce7-...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,16493,Peristylus papuanus,NaN,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1614,38625,Spathoglottis papuana puberiflora,NaN,obscured,ALA_DR493_717,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1751,41410,Thelychiton,NaN,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1760,28167,Thelymitra truncata,NaN,obscured,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
unknownToInat.groupby(['status','new_geoprivacy']).size()

status                 new_geoprivacy
Critically Endangered  obscured            9
                       open               27
Endangered             obscured           12
                       open               52
Extinct                open                2
Extinct in the wild    obscured            1
                       open                9
Least concern          obscured           55
Near Threatened        obscured           12
                       open               48
Special least concern  obscured           87
                       open               64
Vulnerable             obscured           33
                       open              107
dtype: int64

In [24]:
noncomply

,wildnetTaxonID,name,status,wildnet_geoprivacy,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker
231,36927,Calanthe masuca x Calanthe vestita,NaN,obscured,ALA_DR493_227,NaN,NaN,NaN,HYBRID,NaN
265,6156,Chiloglottis sp. (Mango Flat D.L.Jones 2547),Least concern,obscured,ALA_DR493_262,Chiloglottis spec.,Chiloglottis spec.,Chiloglottis spec.,INFORMAL,sp.
294,36585,Corunastylis sp. (Fraser Island NP R.Crane 2063),Special least concern,obscured,ALA_DR652_1392,Corunastylis spec.,Corunastylis spec.,Corunastylis spec.,INFORMAL,sp.
295,36583,Corunastylis sp. (Wyberba D.L.Jones+ 2557),Special least concern,obscured,ALA_DR652_1393,Corunastylis spec.,Corunastylis spec.,Corunastylis spec.,INFORMAL,sp.
332,40864,Dendrobium 'Bunts Bali' x Dendrobium x lavarac...,NaN,obscured,ALA_DR493_329,NaN,NaN,NaN,HYBRID,NaN
333,38443,Dendrobium 'Mauna Kea',NaN,obscured,ALA_DR493_330,Dendrobium cv.,Dendrobium cv.,Dendrobium cv. 'Mauna Kea',CULTIVAR,cv.
353,35005,Dendrobium discolor x D.nindii,Least concern,obscured,ALA_DR493_350,NaN,NaN,NaN,HYBRID,NaN
354,40865,Dendrobium discolor x Dendrobium nindii,Least concern,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,HYBRID,NaN
370,9305,Dendrobium kingianum x D.speciosum,Least concern,obscured,ALA_DR493_367,NaN,NaN,NaN,HYBRID,NaN
381,36930,Dendrobium phalaenopsis var. schroederianum x ...,NaN,obscured,ALA_DR493_378,NaN,NaN,NaN,HYBRID,NaN


In [27]:
pd.concat([noncomply,unknownToInat]).to_csv(sourcedir + "qld-no-inat-taxa-match.csv",index=False)

### iNaturalist statuses unaffected by these changes
Candidates for investigation and removal.

In [ ]:
leftoverstatuses = inatstatuses.merge(updates, how="left", left_on="status_id", right_on="id")#.apply(lambda x: pd.isnull(x['id']))
leftoverstatuses = leftoverstatuses[pd.isnull(leftoverstatuses['id'])]
numleftovers = len(leftoverstatuses.index)
leftoverstatuses = leftoverstatuses[leftoverstatuses['is_active'] != "False"] # synonyms - if it's known to have a synonym that's probably ok
leftoverstatuses = leftoverstatuses[leftoverstatuses['user_id'] == "708886"] # only remove mine
leftoverstatuses = leftoverstatuses[leftoverstatuses['status_x'] == 'endangered'] #quite possibly wrong
numneedingattention = len(leftoverstatuses.index)
leftoverstatuses

In [ ]:
leftoverstatuses.to_csv(sourcedir + "qld-outstanding-inat-statuses.csv")

Notes on these, which should all be manually handled:
https://www.inaturalist.org/taxa/1357800 _Eulophia pulchra_ search on ALA returns _Oeceoclades pelorica_ (or Eulophia pelorica) Vulnerable in Qld
https://www.inaturalist.org/taxa/369238 _Dendrobium mortii_ is known in Qld as _Dockrillia mortii_ with a status of SLC.
https://www.inaturalist.org/taxa/631112 _Trichomanes endlicherianum_ is known as _Polyphlebium endlicherianum_ and on the list with a status of Vulnerable in Qld
... etc

# Stats

In [ ]:
# Stats
numsensitive = len(sensitivelist.index)
numconservation = len(conservationlist.index)
numupdates  = len(updates.index)
numadditions  = len(additions.index)
numnoinatstatus = len(noinatstatus.index)
numunknownToInat = len(unknownToInat.index)
numnoncomply = len(noncomply.index)
numcomply = len(statelist.index)
numdupinfo = len(dupinformation.index)
d = {'State Sensitive List': [numsensitive],
     'State Conservation List': [numconservation],
     'State lists merged': [numfullstatelist],
     'State list species successfully parsed' : [numcomply],
     'State list species not successfully parsed': [numnoncomply],
     'Duplicate/conflicting species statuses': [numdupinfo],
     'State list species matched to iNat status (Updates)': [numupdates],
     'State list species not matched to Inat Status': [numnoinatstatus],
     'iNat Additions (no status but yes taxon)': [numadditions],
     'State list species with no matched status or taxon':[numunknownToInat],
     'iNat statuses not updated': [numleftovers],
     'iNat statuses needing manual attention': [numneedingattention]}

pd.DataFrame(data=d).transpose()

In [ ]:
# should we search for synonyms?
#Litoria lorica
inattaxa[inattaxa['scientificName']=='Ranoidea lorica']
